In [ ]:
# 4.1 Data Cleaning and Transformation

# Import necessary libraries
import pandas as pd

# Define file paths for the datasets
file_paths = {
    'orders': 'olist_orders_dataset.csv',
    'customers': 'olist_customers_dataset.csv',
    'products': 'olist_products_dataset.csv',
    'sellers': 'olist_sellers_dataset.csv',
    'order_items': 'olist_order_items_dataset.csv',
    'order_payments': 'olist_order_payments_dataset.csv',
    'order_reviews': 'olist_order_reviews_dataset.csv'
}

# Load datasets into pandas DataFrames
dataframes = {}
for name, path in file_paths.items():
    df = pd.read_csv(path)
    print(f"Loaded '{name}' with shape: {df.shape[0]} rows and {df.shape[1]} columns")
    dataframes[name] = df

# Define a function to clean each DataFrame

def clean_dataframe(name, df):
    print(f"\nCleaning '{name}' dataset:")
    
    # Inspect missing values
    missing = df.isnull().sum()
    print("Missing values per column:")
    print(missing[missing > 0])
    
    # Drop rows that are completely empty
    df = df.dropna(how='all')
    
    # Fill missing values
    # Categorical columns: fill with 'unknown'
    cat_cols = df.select_dtypes(include=['object']).columns
    for col in cat_cols:
        if df[col].isnull().any():
            df[col].fillna('unknown', inplace=True)
    
    # Numeric columns: fill with 0
    num_cols = df.select_dtypes(include=['number']).columns
    for col in num_cols:
        if df[col].isnull().any():
            df[col].fillna(0, inplace=True)
    
    # Remove duplicate rows
    dup_count = df.duplicated().sum()
    print(f"Found {dup_count} duplicate rows. Dropping duplicates...")
    df = df.drop_duplicates()
    
    print(f"After cleaning '{name}': shape = {df.shape}\n")
    return df

# Apply cleaning to all DataFrames
cleaned_data = {}
for name, df in dataframes.items():
    cleaned_data[name] = clean_dataframe(name, df)

# Display the cleaned 'orders' DataFrame
cleaned_data['orders'].head()

Loaded 'orders' with shape: 99441 rows and 8 columns
Loaded 'customers' with shape: 99441 rows and 5 columns
Loaded 'products' with shape: 32951 rows and 9 columns
Loaded 'sellers' with shape: 3095 rows and 4 columns
Loaded 'order_items' with shape: 112650 rows and 7 columns
Loaded 'order_payments' with shape: 103886 rows and 5 columns
Loaded 'order_reviews' with shape: 99224 rows and 7 columns


In [ ]:
# 4.2 Creating Calculated Columns

# Pull your cleaned DataFrames out of the dict
orders         = cleaned_data['orders']
order_items    = cleaned_data['order_items']
order_payments = cleaned_data['order_payments']

# Convert the orders’ date columns
date_cols = [
    'order_purchase_timestamp',
    'order_approved_at',
    'order_delivered_carrier_date',
    'order_delivered_customer_date'
]

for c in date_cols:
    orders[c] = pd.to_datetime(orders[c], errors='coerce')

# Compute total installments per order
payment_counts = (
    order_payments
      .groupby('order_id')['payment_installments']
      .sum()
      .reset_index(name='payment_count')
)

# Merge dates & payment counts onto each item row
oi = (
    order_items
      .merge(
          orders[[
            'order_id',
            'order_purchase_timestamp',
            'order_delivered_customer_date'
          ]],
          on='order_id',
          how='left'
      )
      .merge(payment_counts, on='order_id', how='left')
)

# Create Total Price
oi['total_price'] = oi['price'] + oi['freight_value']

# Create Delivery Time (in days)
oi['delivery_time'] = (
    oi['order_delivered_customer_date'] 
    - oi['order_purchase_timestamp']
).dt.days

# Create Profit Margin
oi['profit_margin'] = oi['price'] - oi['freight_value']

# Inspect the results
oi[['order_id','price','freight_value','total_price',
    'delivery_time','payment_count','profit_margin']].head()


,order_id,price,freight_value,total_price,delivery_time,payment_count,profit_margin
0,00010242fe8c5a6d1ba2dd792cb16214,58.90,13.29,72.19,7.0,2.0,45.61
1,00018f77f2f0320c557190d7a144bdd3,239.90,19.93,259.83,16.0,3.0,219.97
2,000229ec398224ef6ca0657da4fc703e,199.00,17.87,216.87,7.0,5.0,181.13
3,00024acbcdf0a6daa1e931b038114c75,12.99,12.79,25.78,6.0,2.0,0.20
4,00042b26cf59d7ce69dfabb4e55b4fd9,199.90,18.14,218.04,25.0,3.0,181.76


In [ ]:
# 4.3 Using Window Functions Over Partitions (Pandas)

# Pull cleaned tables
orders         = cleaned_data['orders']
order_items    = cleaned_data['order_items']
order_payments = cleaned_data['order_payments']
products       = cleaned_data['products']

# Convert dates
orders['order_purchase_timestamp']      = pd.to_datetime(orders['order_purchase_timestamp'], errors='coerce')
orders['order_delivered_customer_date'] = pd.to_datetime(orders['order_delivered_customer_date'], errors='coerce')

# Compute total installments per order
payment_counts = (
    order_payments
      .groupby('order_id')['payment_installments']
      .sum()
      .reset_index(name='payment_count')
)

# Build enriched order-items DF
oi = (
    order_items
      .merge(orders[['order_id','order_purchase_timestamp','order_delivered_customer_date']],
             on='order_id', how='left')
      .merge(payment_counts, on='order_id', how='left')
)
oi['delivery_time'] = (oi['order_delivered_customer_date'] - oi['order_purchase_timestamp']).dt.days

# Merge in customer and category
df_win = (
    oi
      .merge(orders[['order_id','customer_id']], on='order_id', how='left')
      .merge(products[['product_id','product_category_name']], on='product_id', how='left')
)

# Running total of price per customer
df_win = df_win.sort_values(['customer_id','order_purchase_timestamp'])
df_win['total_sales_per_customer'] = df_win.groupby('customer_id')['price'].cumsum()

# Rolling avg delivery time per category (window=3)
df_win = df_win.sort_values(['product_category_name','order_purchase_timestamp'])
df_win['rolling_avg_delivery_time'] = (
    df_win
      .groupby('product_category_name')['delivery_time']
      .rolling(window=3, min_periods=1)
      .mean()
      .reset_index(level=0, drop=True)
)

# Inspect
df_win[[
    'order_id','customer_id','price','total_sales_per_customer',
    'product_category_name','delivery_time','rolling_avg_delivery_time'
]].head(10)


,order_id,customer_id,price,total_sales_per_customer,product_category_name,delivery_time,rolling_avg_delivery_time
42783,616f1f539d7add607844e0199fed7ea6,4a3ceccbd2b8304e6aa7c986c1a4f755,21.99,21.99,agro_industria_e_comercio,8.0,8.000000
42784,616f1f539d7add607844e0199fed7ea6,4a3ceccbd2b8304e6aa7c986c1a4f755,21.99,43.98,agro_industria_e_comercio,8.0,8.000000
5813,0d1bbf582326272fa550ed829bd2e1d4,0ef9b2f2eb46a3d1193b12676de597a3,21.99,21.99,agro_industria_e_comercio,8.0,8.000000
21799,31cb8821ab778cd23ebb6ce6f9e2bce0,b3ff37db557452c87c756f90457fe7b1,21.99,21.99,agro_industria_e_comercio,4.0,6.666667
51295,749fcca936a6d6cf6373d2b6f7d72530,2ea8bb2930217f9907af50f779f18386,21.99,21.99,agro_industria_e_comercio,30.0,14.000000
49507,70807adf40446c39c3504cd513bcf3aa,fff200ad56bd3e1caa576cce9d282672,92.90,92.90,agro_industria_e_comercio,16.0,16.666667
105895,f09d0203a51785e9ba9279607c38958c,90abf5b0ac7c7b7bb763da066ca7e60b,21.99,21.99,agro_industria_e_comercio,20.0,22.000000
42454,60b4bdf4f0d6dfa53d78a82a4fbd5a6a,f2a1cf3342383deb7c8def00263da816,21.99,21.99,agro_industria_e_comercio,6.0,14.000000
67769,9af45b618745db1a57fbfbeaceedc889,4e798bde1c7399fbb0763d0f35d9af07,21.99,21.99,agro_industria_e_comercio,3.0,9.666667
29049,4212c4acae39b267aabc4696d873cbcf,80dfe2f69a889e178b3b4d964db72f3d,21.99,21.99,agro_industria_e_comercio,9.0,6.000000


In [ ]:
# 4.4 Saving Processed Data to SQL Server (Fact & Dimension Tables)

import pandas as pd
from sqlalchemy import create_engine, inspect
import urllib

# Load your cleaned DataFrames
orders         = cleaned_data['orders']
order_items    = cleaned_data['order_items']
order_payments = cleaned_data['order_payments']
products       = cleaned_data['products']
sellers        = cleaned_data['sellers']
customers      = cleaned_data['customers']

# Rebuild `oi` with calculated columns

# Convert dates
orders['order_purchase_timestamp']      = pd.to_datetime(
    orders['order_purchase_timestamp'], errors='coerce'
)
orders['order_delivered_customer_date'] = pd.to_datetime(
    orders['order_delivered_customer_date'], errors='coerce'
)

# Sum payment installments per order
payment_counts = (
    order_payments
      .groupby('order_id')['payment_installments']
      .sum()
      .reset_index(name='payment_count')
)

# Merge into order_items
oi = (
    order_items
      .merge(
          orders[[
              'order_id',
              'order_purchase_timestamp',
              'order_delivered_customer_date'
          ]],
          on='order_id', how='left'
      )
      .merge(payment_counts, on='order_id', how='left')
)

# Add your 4.2 calculated columns
oi['total_price']   = oi['price'] + oi['freight_value']
oi['delivery_time'] = (
    oi['order_delivered_customer_date']
    - oi['order_purchase_timestamp']
).dt.days
oi['profit_margin'] = oi['price'] - oi['freight_value']
# payment_count already came in via the merge

# Define Fact & Dimension tables

# Fact: enriched order‐items
fact_order_items = oi[[
    'order_id','order_item_id','product_id','seller_id',
    'price','freight_value','total_price',
    'delivery_time','payment_count','profit_margin'
]].copy()

# Dim: Customers
dim_customers = (
    customers
      .drop_duplicates('customer_id')
      .rename(columns={
          'customer_zip_code_prefix':'zip_prefix',
          'customer_city':'city',
          'customer_state':'state'
      })
)

# Dim: Sellers
dim_sellers = (
    sellers
      .drop_duplicates('seller_id')
      .rename(columns={
          'seller_zip_code_prefix':'zip_prefix',
          'seller_city':'city',
          'seller_state':'state'
      })
)

# Dim: Date (purchase & delivery)
od = orders[[
    'order_purchase_timestamp',
    'order_delivered_customer_date'
]].copy()
od['purchase_date'] = od['order_purchase_timestamp'].dt.date
od['delivery_date'] = od['order_delivered_customer_date'].dt.date

all_dates = pd.concat([
    od[['purchase_date']].rename(columns={'purchase_date':'date'}),
    od[['delivery_date']].rename(columns={'delivery_date':'date'})
]).dropna().drop_duplicates().reset_index(drop=True)

dim_date = all_dates.copy()
dim_date['year']    = pd.DatetimeIndex(dim_date['date']).year
dim_date['month']   = pd.DatetimeIndex(dim_date['date']).month
dim_date['day']     = pd.DatetimeIndex(dim_date['date']).day
dim_date['quarter'] = pd.DatetimeIndex(dim_date['date']).quarter

# Dim: Products – only keep the columns actually present
prod_cols = ['product_id','product_category_name']
# potential extras per spec :contentReference[oaicite:0]{index=0}
extras = [
    'product_name_length','product_description_length',
    'product_photos_qty','product_weight_g',
    'product_length_cm','product_height_cm','product_width_cm',
    # some datasets spell them with 'lenght'
    'product_name_lenght','product_description_lenght'
]
for c in extras:
    if c in products.columns:
        prod_cols.append(c)

dim_products = (
    products
      .drop_duplicates('product_id')
      .loc[:, prod_cols]
      .rename(columns={
          # normalize spelling if needed
          'product_name_lenght':'product_name_length',
          'product_description_lenght':'product_description_length'
      })
)

# Push all to SQL Server

# Make sure your DB (e.g. 'olist_etl') already exists in SSMS
conn_str = urllib.parse.quote_plus(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=LHIND-N211858;"
    "DATABASE=olist_etl;"
    "Trusted_Connection=yes;"
)
engine = create_engine(f"mssql+pyodbc:///?odbc_connect={conn_str}")


for table_name, df in [
    ('fact_order_items', fact_order_items),
    ('dim_customers',    dim_customers),
    ('dim_sellers',      dim_sellers),
    ('dim_date',         dim_date),
    ('dim_products',     dim_products),
]:
    df.to_sql(table_name, con=engine, if_exists='replace', index=False)

# Verify in Python and then refresh in SSMS
inspector = inspect(engine)
print("Created tables:", inspector.get_table_names())


Created tables: ['dim_customers', 'dim_date', 'dim_products', 'dim_sellers', 'fact_order_items']
